In [34]:
import re
import json
from io import BytesIO

import requests
import pandas as pd
from PIL import Image

In [10]:
def load_img(url: str):
    res = requests.get(url)
    img = Image.open(BytesIO(res.content))
    
    return img

In [2]:
with open('./data/semi_struct_ocr/홍대 회식 맛집.json', 'r', encoding='utf-8') as f:
    blog = json.load(f)

In [28]:
ad_keywords = ["내돈내산",
"지원을받아",
"지원받아",
"후원을받아",
"후원받아",
"원고료",
"체험단",
"제공받아",
"금전적지원",
"사비로",
"업체로부터",
"협찬",
"제품또는서비스를",
"업체에서",
"소정의",
"고료를",
"할인을",
"레뷰",
"강남오빠",
"디너의여왕",
"서울오빠",
"이놀자데뷰"]

In [43]:
HAN_REGEX = re.compile('[^가-힣]')

In [46]:
df = []
for article in blog:
    easy_text = ""
    tess_text = ""
    for p in article['contents']:
        if p['info'] not in ['img', 'gif']: continue
        
        if p.get('tess_text'):
            easy_text += p['easy_text']
            tess_text += p["tess_text"]
            
    df.append({
        "article_id": article["id"],
        "easyocr": HAN_REGEX.sub('', easy_text),
        "tesseractocr": HAN_REGEX.sub('', tess_text)
    })

In [47]:
df = pd.DataFrame(df)
df

,article_id,easyocr,tesseractocr
0,shuenmama.223027650182,장안집쟁반집돼지고기활삼심군파심수부위소고기우상적삼기심감비살모툼정반우궁반우산경기비살실치...,벌집업데기막창사이트맥주때심술도도청하이캐삐때맛깔스러운잔반찬본포스팅은업체로부터서비스만을
1,rosepink1974.223153722255,로잠립흐로중위이외로그소계청년히로대내리신드규시간이비법입니다도래위다이총의냉난방중하문올곡...,범는는트는는가자허오므수으로이이이오으로오고아노겨분그오그오이개이아제도이기으으에오미쿄록대...
2,mou25.223209216526,,
3,lulu_l.223118434610,올호석듣데이연막단매스럽이극찬한생방송연막람트데이년월일발영리얼맛집연랑불에구위야제맛연단찢...,는노털로오느고기류대표메뉴사구시그니쳐고을수스대오개사이드벡씨네색후토마끼나으윤씨네녕면물비...
4,ruston_.223161590597,팔로우여기논회사회식할다자주오는데올더가다만족하고갑니다등심은들튼이고모든고기가정말맛있어요...,브쁘으설명도친절하게셔서취향에맞게잘특별한날부모님이도너무너무좋은한우특별고깃집문할게요메뉴...
...,...,...,...
883,r4race.223079109532,로숲돈욕문욕기본에충실이켜습니다우리움에서나고자라선한진로로만든해국내산콩과소공으로직절만들...,므노고그이배게나사오나테그나나노녀누너벼저애애자고스애스나원나사내어원자원초계비빔면이글은레...
884,hook415.220707753175,,
885,highkey_today.223070887072,구이흉대오임크빵훨다네이버불로그사일기네이버불로그도시인기오진수씨고기맨프하곡소중하사주세요...,애드온밸기투구거홍대오잉크파는반합으로고기와할께세공되는파는빈함뚜경이익허주셔요고거는자주고...
886,gogoitis.223059408847,흙통피푹틀흉홍콩포자방다좁패습주세중콤포사콩포차양흥콩포차춤경주문내역초후에닫히니다닫기마지...,비무쁜매드글대나그개고으니미는을다아그뷰더다더너결호주개블배노가구가오오내뼈오거도이개백개매...


,article_id,easyocr,tesseractocr


In [51]:
for kywd in ad_keywords:
    df.loc[df[df.easyocr.str.contains(kywd)].index, 'label'] = 1

In [53]:
df['label'] = df.label.fillna(0)
df

,article_id,easyocr,tesseractocr,label
0,shuenmama.223027650182,장안집쟁반집돼지고기활삼심군파심수부위소고기우상적삼기심감비살모툼정반우궁반우산경기비살실치...,벌집업데기막창사이트맥주때심술도도청하이캐삐때맛깔스러운잔반찬본포스팅은업체로부터서비스만을,1.0
1,rosepink1974.223153722255,로잠립흐로중위이외로그소계청년히로대내리신드규시간이비법입니다도래위다이총의냉난방중하문올곡...,범는는트는는가자허오므수으로이이이오으로오고아노겨분그오그오이개이아제도이기으으에오미쿄록대...,1.0
2,mou25.223209216526,,,0.0
3,lulu_l.223118434610,올호석듣데이연막단매스럽이극찬한생방송연막람트데이년월일발영리얼맛집연랑불에구위야제맛연단찢...,는노털로오느고기류대표메뉴사구시그니쳐고을수스대오개사이드벡씨네색후토마끼나으윤씨네녕면물비...,0.0
4,ruston_.223161590597,팔로우여기논회사회식할다자주오는데올더가다만족하고갑니다등심은들튼이고모든고기가정말맛있어요...,브쁘으설명도친절하게셔서취향에맞게잘특별한날부모님이도너무너무좋은한우특별고깃집문할게요메뉴...,1.0
...,...,...,...,...
883,r4race.223079109532,로숲돈욕문욕기본에충실이켜습니다우리움에서나고자라선한진로로만든해국내산콩과소공으로직절만들...,므노고그이배게나사오나테그나나노녀누너벼저애애자고스애스나원나사내어원자원초계비빔면이글은레...,1.0
884,hook415.220707753175,,,0.0
885,highkey_today.223070887072,구이흉대오임크빵훨다네이버불로그사일기네이버불로그도시인기오진수씨고기맨프하곡소중하사주세요...,애드온밸기투구거홍대오잉크파는반합으로고기와할께세공되는파는빈함뚜경이익허주셔요고거는자주고...,0.0
886,gogoitis.223059408847,흙통피푹틀흉홍콩포자방다좁패습주세중콤포사콩포차양흥콩포차춤경주문내역초후에닫히니다닫기마지...,비무쁜매드글대나그개고으니미는을다아그뷰더다더너결호주개블배노가구가오오내뼈오거도이개백개매...,0.0


In [56]:
df[df.label == 0]

,article_id,easyocr,tesseractocr,label
2,mou25.223209216526,,,0.0
3,lulu_l.223118434610,올호석듣데이연막단매스럽이극찬한생방송연막람트데이년월일발영리얼맛집연랑불에구위야제맛연단찢...,는노털로오느고기류대표메뉴사구시그니쳐고을수스대오개사이드벡씨네색후토마끼나으윤씨네녕면물비...,0.0
5,ani_i.223118572729,그로예국난점심묵선분듬갈국수사부째버섯갈국수인분인부터맥기조그가능간장소스제공엔순제대만식퇴...,,0.0
7,jk01128.223201857381,고주수넘마꽃살집소고기모로구기민에시간의기다림숙성소고기전문점하나온전한소고기의깊은향과맛울...,나온전한소고기의깊은티얼마프라임와규한우만을사용합니나가던나기가히로오더하고드가항과열이뛰어...,0.0
8,aoekdud.223100176598,논연주람가리옆막탑연막단년각본현택로돌풀그진원겁연막탑연막탓기기월니추더마다민프로미드먼막닭...,,0.0
...,...,...,...,...
882,sky3545.221079400441,,,0.0
884,hook415.220707753175,,,0.0
885,highkey_today.223070887072,구이흉대오임크빵훨다네이버불로그사일기네이버불로그도시인기오진수씨고기맨프하곡소중하사주세요...,애드온밸기투구거홍대오잉크파는반합으로고기와할께세공되는파는빈함뚜경이익허주셔요고거는자주고...,0.0
886,gogoitis.223059408847,흙통피푹틀흉홍콩포자방다좁패습주세중콤포사콩포차양흥콩포차춤경주문내역초후에닫히니다닫기마지...,비무쁜매드글대나그개고으니미는을다아그뷰더다더너결호주개블배노가구가오오내뼈오거도이개백개매...,0.0
